In [29]:
import pandas as pd
import json
import os
import matplotlib.pyplot as plt

In [2]:
def read_merge_json(df, json_path):
    filelist = os.listdir(json_path)
    filelist.sort()
    for file in filelist:
        final_path = os.path.join(json_path, file)
        with open(final_path, 'r') as f:
            prices = json.load(f)
        date = file.split("price-history-")[1].split(".txt")[0].replace("-", "")
        prices = pd.DataFrame.from_dict(prices, orient='index', columns=[date])
        df = df.merge(prices, left_index=True, right_index=True)
    return df

In [44]:
def plot_card_price(df, name=None, cardset=None, foil=0, cardID=None):
    if cardID is not None:
        card_prices = df.loc[cardID]
    else:
        card_prices = df.loc[(df['name'] == name) & (df['cardset'] == cardset) & (df['foil'] == foil)]
    card_prices = card_prices.drop(['name', 'cardset', 'rarity', 'foil'], axis=1)
    print(card_prices.index)
    dates = pd.to_datetime(card_prices.index)
    # Plotting
    plt.figure(figsize=(10, 6))  # Set the figure size for better readability
    plt.plot(dates, card_prices.values, marker='o', linestyle='-')  # Plot with markers for each data point

    plt.title(f"Price over time for cardID {cardID}" if cardID is not None else f"Price over time for {name} ({cardset}) {'Foil' if foil else ''}")
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.xticks(rotation=45)  # Rotate date labels for better readability
    plt.tight_layout()  # Adjust layout to make room for the rotated date labels
    plt.show()

In [45]:
plot_card_price(df, name="Phyrexian Dreadnought", cardset="MI", foil=0)

Index(['7185'], dtype='object')


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 7185, at position 0

In [3]:
path_card_definitions = os.path.join("0_data", "0_card_definitions", "card-definitions.txt")
with open(path_card_definitions) as f:
    card_definitions = json.load(f)
df_cards = pd.DataFrame.from_dict(card_definitions, orient='index')

In [10]:
path_card_prices = os.path.join("0_data", "1_card_prices")
df = read_merge_json(df_cards, path_card_prices)

In [11]:
filters = {
    "name": "Phyrexian Dreadnought",
    "cardset": "MI",
    "foil": 0
}
df_filtered = df.copy()
for filter in filters:
    df_filtered = df_filtered[df_filtered[filter] == filters[filter]]
df_filtered

,name,cardset,rarity,foil,20240101,20240102,20240103,20240104,20240105,20240106,...,20240615,20240616,20240617,20240618,20240619,20240620,20240621,20240622,20240623,20240624
7185,Phyrexian Dreadnought,MI,Rare,0,8.18,7.39,7.39,7.39,7.83,8.57,...,14.51,14.51,14.9,15.24,15.24,17.02,17.99,17.99,17.99,17.57


In [18]:
pd.to_datetime(df.loc["7185"].index[4:])

DatetimeIndex(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',
               '2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08',
               '2024-01-09', '2024-01-10',
               ...
               '2024-06-15', '2024-06-16', '2024-06-17', '2024-06-18',
               '2024-06-19', '2024-06-20', '2024-06-21', '2024-06-22',
               '2024-06-23', '2024-06-24'],
              dtype='datetime64[ns]', length=176, freq=None)

In [9]:
df.index

Index(['7185'], dtype='object')

In [9]:

# df_prices = pd.DataFrame.from_dict(price_history, orient='index', columns=['price'])

In [14]:
df = df_cards.merge(df_prices, left_index=True, right_index=True)
df

,name,cardset,rarity,foil,price
127558,Modern Horizons 3 Booster,MH3,Booster,0,5.030
127520,"Avatar - Shilgengar, Sire of Famine",VAN,Rare,0,0.006
127516,Endurance,SPG,Mythic,1,3.120
127514,Fury,SPG,Mythic,1,0.290
127512,Grief,SPG,Mythic,1,10.800
...,...,...,...,...,...
19,"Avatar - Akroma, Angel of Wrath",VAN,Rare,0,0.006
18,Avatar - Phage the Untouchable,VAN,Rare,0,0.080
14,Avatar - Flametongue Kavu,VAN,Rare,0,0.007
13,Avatar - Fallen Angel,VAN,Rare,0,0.200


In [20]:
df = df_cards.join(df_prices)
df

,name,cardset,rarity,foil,price
127558,Modern Horizons 3 Booster,MH3,Booster,0,5.030
127520,"Avatar - Shilgengar, Sire of Famine",VAN,Rare,0,0.006
127516,Endurance,SPG,Mythic,1,3.120
127514,Fury,SPG,Mythic,1,0.290
127512,Grief,SPG,Mythic,1,10.800
...,...,...,...,...,...
19,"Avatar - Akroma, Angel of Wrath",VAN,Rare,0,0.006
18,Avatar - Phage the Untouchable,VAN,Rare,0,0.080
14,Avatar - Flametongue Kavu,VAN,Rare,0,0.007
13,Avatar - Fallen Angel,VAN,Rare,0,0.200


In [21]:
df[(df["name"]=="Phyrexian Dreadnought") & (df["foil"]==0)]

,name,cardset,rarity,foil,price
43534,Phyrexian Dreadnought,PRM,Rare,0,0.07
7185,Phyrexian Dreadnought,MI,Rare,0,17.99
